# Fetching air quality data from the FMI open data timeseries API

[API documentation](https://github.com/fmidev/smartmet-plugin-timeseries/blob/master/docs/Using-the-Timeseries-API.md),
[API examples](https://github.com/fmidev/smartmet-plugin-timeseries/blob/master/docs/Examples.md),
[JSON API example call](https://opendata.fmi.fi/timeseries?format=json&groupareas=0&producer=airquality_urban&area=Helsinki&param=time,fmisid,PM10_PT1H_avg,PM25_PT1H_avg,O3_PT1H_avg,CO_PT1H_avg,SO2_PT1H_avg,NO2_PT1H_avg,TRSC_PT1H_avg),
[CSV API call for the fmisid to name mapping](https://opendata.fmi.fi/timeseries?format=ascii&groupareas=0&separator=,&producer=airquality_urban&area=Finland&param=fmisid,name,latitude,longitude&starttime=2022-08-26T08:00:00%2B00:00&endtime=2022-08-26T08:00:00%2B00:00&tz=UTC).

In [4]:
import requests
import pendulum
import pandas as pd
import numpy as np

In [5]:
start_time = pendulum.yesterday('UTC')
start_time = pendulum.now('UTC').subtract(days=5)
end_time = pendulum.tomorrow('UTC')

aq_fields = {
    'fmisid': np.int32,
    'time': np.datetime64,
    'AQINDEX_PT1H_avg': np.float64,
    'PM10_PT1H_avg': np.float64,
    'PM25_PT1H_avg': np.float64,
    'O3_PT1H_avg': np.float64,
    'CO_PT1H_avg': np.float64,
    'SO2_PT1H_avg': np.float64,
    'NO2_PT1H_avg': np.float64,
    'TRSC_PT1H_avg': np.float64,
}

url = 'https://opendata.fmi.fi/timeseries'

params = {
    'format': 'json',
    'precision': 'double',
    'groupareas': '0',
    'producer': 'airquality_urban',
    'area': 'Uusimaa',
    'param': ','.join(aq_fields.keys()),
    'starttime': start_time.isoformat(timespec="seconds"),
    'endtime': end_time.isoformat(timespec="seconds"),
    'tz': 'UTC',
}

data = requests.get(url, params=params).json()

In [6]:
df = pd.DataFrame(data).astype(aq_fields)
df = df.set_index(['fmisid', 'time'])
df[0:10]

AQINDEX_PT1H_avg  PM10_PT1H_avg  PM25_PT1H_avg  \
fmisid time                                                                  
100662 2022-09-08 11:00:00               1.0            8.2            3.7   
       2022-09-08 12:00:00               2.0            6.2            4.2   
       2022-09-08 13:00:00               2.0            8.6            3.1   
       2022-09-08 14:00:00               2.0            8.8            3.5   
       2022-09-08 15:00:00               1.0            5.7            3.7   
       2022-09-08 16:00:00               2.0            5.8            1.8   
       2022-09-08 17:00:00               1.0            6.2            3.6   
       2022-09-08 18:00:00               1.0            5.6            4.2   
       2022-09-08 19:00:00               1.0            7.2            4.7   
       2022-09-08 20:00:00               1.0            7.3            4.2   

                            O3_PT1H_avg  CO_PT1H_avg  SO2_PT1H_avg  \
fmisid time                                                          
100662 2022-09-08 11:00:00         58.3          NaN           1.6   
       2022-09-08 12:00:00         61.4          NaN           1.2   
       2022-09-08 13:00:00         61.5          NaN           1.5   
       2022-09-08 14:00:00         61.1          NaN           1.4   
       2022-09-08 15:00:00         58.6          NaN           1.3   
       2022-09-08 16:00:00         60.6          NaN           1.3   
       2022-09-08 17:00:00         59.0          NaN           1.1   
       2022-09-08 18:00:00         48.8          NaN           1.3   
       2022-09-08 19:00:00         47.5          NaN           1.2   
       2022-09-08 20:00:00         50.1          NaN           1.0   

                            NO2_PT1H_avg  TRSC_PT1H_avg  
fmisid time                                              
100662 2022-09-08 11:00:00           6.1            NaN  
       2022-09-08 12:00:00           3.9            NaN  
       2022-09-08 13:00:00           4.6            NaN  
       2022-09-08 14:00:00           6.4            NaN  
       2022-09-08 15:00:00           9.2            NaN  
       2022-09-08 16:00:00           6.8            NaN  
       2022-09-08 17:00:00           8.0            NaN  
       2022-09-08 18:00:00          16.1            NaN  
       2022-09-08 19:00:00          15.6            NaN  
       2022-09-08 20:00:00          11.4            NaN

In [7]:
df.to_parquet('data/airquality.parquet', compression='zstd')